1. mataplotlib -> 시각화
2. pandas -> 표 형태로 데이터를 다룰 때 -> 전처리 과정(결측치, 스케일링[cm -> m], 컬럼병합/추출)
3. numpy -> 배열과 같은 기능


시각화의 이유
1. 데이터 탐색(DEA)
2. 상대방에게 결과를 전달하기 위해

## 서울시 구별 CCTV 현황 분석하기
- 서울시 각 구별 CCTV수를 파악
- 인구대비 CCTV비율 파악해서 순위작성
- 인구대비 CCTV예측치 확인 및 부족한 구역파악

In [3]:
import numpy as np
import matplotlib.pyplot as plot
from matplotlib import rc
rc('font', family = 'Malgun Gothic')
import pandas as pd
pd.set_option('display.max_rows', 1000)      # 데이터 row 수 1000개 까지
pd.set_option('display.max_columns', 1000)   # 데이터의 column 수를 1000개 까지
pd.set_option('display.max_colwidth', -1)    # 힌 column의 너비를 데이터 크기에 맞게


C:\Users\smhrd\anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


## 1. CSV파일 읽기 - 서울시 구별 CCTV현황

In [6]:
CCTV_Seoul = pd.read_csv("./Data/CCTV_in_Seoul.csv", encoding = 'utf-8')  
# .은 현재폴더, ..은 현재보다 상위폴더의미

In [10]:
CCTV_Seoul.head(3)  # 위부터 데이터를 가져옴 기본 5개

,기관명,소계,2013년도 이전,2014년,2015년,2016년
0,강남구,2780,1292,430,584,932
1,강동구,773,379,99,155,377
2,강북구,748,369,120,138,204


In [12]:
CCTV_Seoul.tail()   # 아래부터 데이터를 가져옴 기본 5개

,기관명,소계,2013년도 이전,2014년,2015년,2016년
20,용산구,1624,1368,218,112,398
21,은평구,1873,1138,224,278,468
22,종로구,1002,464,314,211,630
23,중구,671,413,190,72,348
24,중랑구,660,509,121,177,109


## 2.엑셀파일읽기 - 서울시 인구현황

In [13]:
pop_Seoul = pd.read_excel("./Data/population_in_Seoul.xls")
pop_Seoul.head()

,기간,자치구,세대,인구,인구.1,인구.2,인구.3,인구.4,인구.5,인구.6,인구.7,인구.8,세대당인구,65세이상고령자
0,기간,자치구,세대,합계,합계,합계,한국인,한국인,한국인,등록외국인,등록외국인,등록외국인,세대당인구,65세이상고령자
1,기간,자치구,세대,계,남자,여자,계,남자,여자,계,남자,여자,세대당인구,65세이상고령자
2,2017.1/4,합계,4202888,10197604,5000005,5197599,9926968,4871560,5055408,270636,128445,142191,2.36,1321458
3,2017.1/4,종로구,72654,162820,79675,83145,153589,75611,77978,9231,4064,5167,2.11,25425
4,2017.1/4,중구,59481,133240,65790,67450,124312,61656,62656,8928,4134,4794,2.09,20764


#### column에 문제가 있기 때문에 옵션으로 설정
- header : 읽고 싶은 row index(0부터 시작)
- usecols : 읽고 싶은 column 선택

In [15]:
pop_Seoul = pd.read_excel("./Data/population_in_Seoul.xls", header = 2, usecols = 'B, D, G, J, N')
pop_Seoul.head()

,자치구,계,계.1,계.2,65세이상고령자
0,합계,10197604.0,9926968.0,270636.0,1321458.0
1,종로구,162820.0,153589.0,9231.0,25425.0
2,중구,133240.0,124312.0,8928.0,20764.0
3,용산구,244203.0,229456.0,14747.0,36231.0
4,성동구,311244.0,303380.0,7864.0,39997.0


In [17]:
pop_Seoul.columns = ['구', '인구수' ,'한국인', '외국인', '고령자']

In [18]:
pop_Seoul.head()

,구,인구수,한국인,외국인,고령자
0,합계,10197604.0,9926968.0,270636.0,1321458.0
1,종로구,162820.0,153589.0,9231.0,25425.0
2,중구,133240.0,124312.0,8928.0,20764.0
3,용산구,244203.0,229456.0,14747.0,36231.0
4,성동구,311244.0,303380.0,7864.0,39997.0


In [21]:
# 컬럼 중 일부만 뽑아서 바꾸는 rename
CCTV_Seoul = CCTV_Seoul.rename(columns={CCTV_Seoul.columns[0] : "구"})
CCTV_Seoul.head()

,구,소계,2013년도 이전,2014년,2015년,2016년
0,강남구,2780,1292,430,584,932
1,강동구,773,379,99,155,377
2,강북구,748,369,120,138,204
3,강서구,884,388,258,184,81
4,관악구,1496,846,260,390,613


## 3. 서울시의 각 구별 CCTV 수 파악하기

In [22]:
CCTV_Seoul

,구,소계,2013년도 이전,2014년,2015년,2016년
0,강남구,2780,1292,430,584,932
1,강동구,773,379,99,155,377
2,강북구,748,369,120,138,204
3,강서구,884,388,258,184,81
4,관악구,1496,846,260,390,613
5,광진구,707,573,78,53,174
6,구로구,1561,1142,173,246,323
7,금천구,1015,674,51,269,354
8,노원구,1265,542,57,451,516
9,도봉구,485,238,159,42,386


In [32]:
CCTV_Seoul.sort_values(by = "소계").head()

,구,소계,2013년도 이전,2014년,2015년,2016년
9,도봉구,485,238,159,42,386
12,마포구,574,314,118,169,379
17,송파구,618,529,21,68,463
24,중랑구,660,509,121,177,109
23,중구,671,413,190,72,348


In [35]:
CCTV_Seoul.sort_values(by = "소계", ascending = False).head()

,구,소계,2013년도 이전,2014년,2015년,2016년
0,강남구,2780,1292,430,584,932
18,양천구,2034,1843,142,30,467
14,서초구,1930,1406,157,336,398
21,은평구,1873,1138,224,278,468
20,용산구,1624,1368,218,112,398


## 인구수대비 CCTV현황 파악하기
- 데이터 병합 -> 양쪽 데;이터에 구 정보가 똑같은지 확인하기

In [36]:
CCTV_Seoul['구'].unique()

array(['강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구', '노원구',
       '도봉구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구',
       '양천구', '영등포구', '용산구', '은평구', '종로구', '중구', '중랑구'], dtype=object)

In [37]:
pop_Seoul['구'].unique()

array(['합계', '종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구',
       '강북구', '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구',
       '금천구', '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구', nan],
      dtype=object)

In [38]:
len(CCTV_Seoul['구'].unique())

25

In [39]:
len(pop_Seoul['구'].unique())

27

- 인구수 데이터에서 구 정보의 '합계' 데이터 삭제
- 인구수 데이터에서 결측치 삭제

In [40]:
pop_Seoul = pop_Seoul.drop([0])

In [41]:
pop_Seoul

,구,인구수,한국인,외국인,고령자
1,종로구,162820.0,153589.0,9231.0,25425.0
2,중구,133240.0,124312.0,8928.0,20764.0
3,용산구,244203.0,229456.0,14747.0,36231.0
4,성동구,311244.0,303380.0,7864.0,39997.0
5,광진구,372164.0,357211.0,14953.0,42214.0
6,동대문구,369496.0,354079.0,15417.0,54173.0
7,중랑구,414503.0,409882.0,4621.0,56774.0
8,성북구,461260.0,449773.0,11487.0,64692.0
9,강북구,330192.0,326686.0,3506.0,54813.0
10,도봉구,348646.0,346629.0,2017.0,51312.0


In [42]:
# 불리언 색인 -> 결측치 찾기
pop_Seoul[pop_Seoul['구'].isnull()]

,구,인구수,한국인,외국인,고령자
26,NaN,NaN,NaN,NaN,NaN


In [43]:
pop_Seoul = pop_Seoul.drop([26])

In [44]:
pop_Seoul

,구,인구수,한국인,외국인,고령자
1,종로구,162820.0,153589.0,9231.0,25425.0
2,중구,133240.0,124312.0,8928.0,20764.0
3,용산구,244203.0,229456.0,14747.0,36231.0
4,성동구,311244.0,303380.0,7864.0,39997.0
5,광진구,372164.0,357211.0,14953.0,42214.0
6,동대문구,369496.0,354079.0,15417.0,54173.0
7,중랑구,414503.0,409882.0,4621.0,56774.0
8,성북구,461260.0,449773.0,11487.0,64692.0
9,강북구,330192.0,326686.0,3506.0,54813.0
10,도봉구,348646.0,346629.0,2017.0,51312.0


### 구 정보가 같은지 확인하기 위해 집합연산 활용 ( set )

In [45]:
CCTV_set = set(CCTV_Seoul['구'].unique())
pop_set = set(pop_Seoul['구'].unique())

In [46]:
CCTV_set - pop_set

set()

In [47]:
pop_set - CCTV_set

set()

#### - 데이터 병합

In [48]:
data_result = pd.merge(CCTV_Seoul, pop_Seoul, on = '구')
data_result.head()

,구,소계,2013년도 이전,2014년,2015년,2016년,인구수,한국인,외국인,고령자
0,강남구,2780,1292,430,584,932,570500.0,565550.0,4950.0,63167.0
1,강동구,773,379,99,155,377,453233.0,449019.0,4214.0,54622.0
2,강북구,748,369,120,138,204,330192.0,326686.0,3506.0,54813.0
3,강서구,884,388,258,184,81,603772.0,597248.0,6524.0,72548.0
4,관악구,1496,846,260,390,613,525515.0,507203.0,18312.0,68082.0


In [49]:
data_result.shape

(25, 10)

- 인구수 대비 CCTV가 많은 지역과 적은지역 확인

In [62]:
# featurn engineering : 데이터의 특성을 가지고 여러 연산을 해서 데이터를 가져오는 것
data_result['인구수대비_CCTV비율'] = data_result['소계'] / data_result['인구수']

In [65]:
data_result.sort_values(by = '인구수대비_CCTV비율', ascending = False).head()

,구,소계,2013년도 이전,2014년,2015년,2016년,인구수,한국인,외국인,고령자,인구수대비_CCTV비율
20,용산구,1624,1368,218,112,398,244203.0,229456.0,14747.0,36231.0,0.006650
22,종로구,1002,464,314,211,630,162820.0,153589.0,9231.0,25425.0,0.006154
23,중구,671,413,190,72,348,133240.0,124312.0,8928.0,20764.0,0.005036
0,강남구,2780,1292,430,584,932,570500.0,565550.0,4950.0,63167.0,0.004873
14,서초구,1930,1406,157,336,398,450310.0,445994.0,4316.0,51733.0,0.004286


## 5. 머신러닝을 통해 예측치를 확인하고 부족한 구를 확인

In [67]:
data_result = data_result.set_index('구')
data_result.head()

,소계,2013년도 이전,2014년,2015년,2016년,인구수,한국인,외국인,고령자,인구수대비_CCTV비율
구,,,,,,,,,,
강남구,2780,1292,430,584,932,570500.0,565550.0,4950.0,63167.0,0.004873
강동구,773,379,99,155,377,453233.0,449019.0,4214.0,54622.0,0.001706
강북구,748,369,120,138,204,330192.0,326686.0,3506.0,54813.0,0.002265
강서구,884,388,258,184,81,603772.0,597248.0,6524.0,72548.0,0.001464
관악구,1496,846,260,390,613,525515.0,507203.0,18312.0,68082.0,0.002847


In [68]:
# 상관계수
data_result.corr()

,소계,2013년도 이전,2014년,2015년,2016년,인구수,한국인,외국인,고령자,인구수대비_CCTV비율
소계,1.000000,0.862756,0.450062,0.624402,0.593398,0.306342,0.304287,-0.023786,0.255196,0.575784
2013년도 이전,0.862756,1.000000,0.121888,0.257748,0.355482,0.168177,0.163142,0.048973,0.105379,0.577185
2014년,0.450062,0.121888,1.000000,0.312842,0.415387,0.027040,0.025005,0.027325,0.010233,0.399458
2015년,0.624402,0.257748,0.312842,1.000000,0.513767,0.368912,0.363796,0.013301,0.372789,0.207860
2016년,0.593398,0.355482,0.415387,0.513767,1.000000,0.144959,0.145966,-0.042688,0.065784,0.438115
인구수,0.306342,0.168177,0.027040,0.368912,0.144959,1.000000,0.998061,-0.153371,0.932667,-0.514143
한국인,0.304287,0.163142,0.025005,0.363796,0.145966,0.998061,1.000000,-0.214576,0.931636,-0.513169
외국인,-0.023786,0.048973,0.027325,0.013301,-0.042688,-0.153371,-0.214576,1.000000,-0.155381,0.079211
고령자,0.255196,0.105379,0.010233,0.372789,0.065784,0.932667,0.931636,-0.155381,1.000000,-0.541024
인구수대비_CCTV비율,0.575784,0.577185,0.399458,0.207860,0.438115,-0.514143,-0.513169,0.079211,-0.541024,1.000000


- 인구대비 CCTV비율 예측값 만들기 with 선형회귀
- 선형회귀 : 데이터를 가장 잘 설명(예측)할 수 있는 수학공식을 만들어내는 모델

In [69]:
from sklearn.linear_model import LinearRegression  # 선형회귀 알고리즘 클래스

In [70]:
linear_model = LinearRegression()

In [79]:
linear_model.fit(data_result[['인구수']], data_result['소계'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [80]:
linear_model.coef_

array([0.00130916])

In [81]:
linear_model.intercept_

645.0664965847219

선형회귀를 통해 나온 공식
y = 0.00130916X + 645.06649658

- 예측값 만들기

In [82]:
predict_CCTV = linear_model.predict(data_result[['인구수']])

In [83]:
predict_CCTV

array([1391.94464487, 1238.42289235, 1077.34202601, 1435.50315451,
       1333.05189553, 1132.29026377, 1231.40708166,  979.01070661,
       1390.48361768, 1101.50134126, 1128.79741381, 1185.12289226,
       1155.18099895, 1073.3765678 , 1234.59620553, 1052.53598367,
       1248.93155299, 1518.91131175, 1273.43648757, 1172.64001207,
        964.76830981, 1292.30154299,  858.22460369,  819.4995281 ,
       1187.71896477])

In [86]:
data_result['오차'] = data_result['소계'] - predict_CCTV

# 양수는 선형모델의 예측보다 더 많이 설치, 음수는 예측보다 부족한 상태를 의미

In [88]:
# CCTV가 예측값보다 부족한 5개 지역
data_result.sort_values(by = '오차').head()

,소계,2013년도 이전,2014년,2015년,2016년,인구수,한국인,외국인,고령자,인구수대비_CCTV비율,오차
구,,,,,,,,,,,
송파구,618,529,21,68,463,667483.0,660584.0,6899.0,72506.0,0.000926,-900.911312
도봉구,485,238,159,42,386,348646.0,346629.0,2017.0,51312.0,0.001391,-616.501341
마포구,574,314,118,169,379,389649.0,378566.0,11083.0,48765.0,0.001473,-581.180999
강서구,884,388,258,184,81,603772.0,597248.0,6524.0,72548.0,0.001464,-551.503155
중랑구,660,509,121,177,109,414503.0,409882.0,4621.0,56774.0,0.001592,-527.718965


In [89]:
# CCTV가 예측값보다 과도한 5개 지역
data_result.sort_values(by = '오차', ascending= False).head()

,소계,2013년도 이전,2014년,2015년,2016년,인구수,한국인,외국인,고령자,인구수대비_CCTV비율,오차
구,,,,,,,,,,,
강남구,2780,1292,430,584,932,570500.0,565550.0,4950.0,63167.0,0.004873,1388.055355
양천구,2034,1843,142,30,467,479978.0,475949.0,4029.0,52975.0,0.004238,760.563512
서초구,1930,1406,157,336,398,450310.0,445994.0,4316.0,51733.0,0.004286,695.403794
용산구,1624,1368,218,112,398,244203.0,229456.0,14747.0,36231.0,0.006650,659.231690
은평구,1873,1138,224,278,468,494388.0,489943.0,4445.0,72334.0,0.003789,580.698457
